# Análisis de sentimiento: pueblos mágicos de México

## Importación de librerias

Usaremos spacy por su eficiencia y por su utilidad en el procesamiento del lenguaje natural en español.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from scipy.stats import randint
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pickle

## Carga de datos

In [18]:
df = pd.read_csv("conjunto_de_datos/Rest-Mex_2025_train.csv", encoding="UTF-8")

## Inspección

In [19]:
df.head()

,Title,Review,Polarity,Town,Region,Type
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,5.0,Sayulita,Nayarit,Restaurant
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",4.0,Tulum,QuintanaRoo,Attractive
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",3.0,Tulum,QuintanaRoo,Hotel
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",4.0,Isla_Mujeres,QuintanaRoo,Attractive
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,5.0,Patzcuaro,Michoacan,Hotel


In [20]:
df.shape

(208051, 6)

In [21]:
df.dtypes

Title        object
Review       object
Polarity    float64
Town         object
Region       object
Type         object
dtype: object

In [22]:
df.isnull().sum()

Title       2
Review      0
Polarity    0
Town        0
Region      0
Type        0
dtype: int64

In [23]:
df['Polarity'].unique()

array([5., 4., 3., 1., 2.])

In [24]:
df['Town'].unique()

array(['Sayulita', 'Tulum', 'Isla_Mujeres', 'Patzcuaro', 'Palenque',
       'Valle_de_Bravo', 'Ixtapan_de_la_Sal', 'Creel', 'Taxco',
       'Valladolid', 'Izamal', 'San_Cristobal_de_las_Casas', 'Atlixco',
       'Tequisquiapan', 'Ajijic', 'Teotihuacan', 'Tequila', 'Bacalar',
       'TodosSantos', 'Parras', 'Coatepec', 'Huasca_de_Ocampo',
       'Tepoztlan', 'Cholula', 'Cuatro_Cienegas', 'Metepec', 'Loreto',
       'Orizaba', 'Tlaquepaque', 'Cuetzalan', 'Bernal', 'Xilitla',
       'Malinalco', 'Real_de_Catorce', 'Chiapa_de_Corzo', 'Mazunte',
       'Tepotzotlan', 'Zacatlan', 'Dolores_Hidalgo', 'Tapalpa'],
      dtype=object)

In [25]:
df['Region'].unique()

array(['Nayarit', 'QuintanaRoo', 'Michoacan', 'Chiapas',
       'Estado_de_Mexico', 'Chihuahua', 'Guerrero', 'Yucatan', 'Puebla',
       'Queretaro', 'Jalisco', 'Baja_CaliforniaSur', 'Coahuila',
       'Veracruz', 'Hidalgo', 'Morelos', 'San_Luis_Potosi', 'Oaxaca',
       'Guanajuato'], dtype=object)

In [26]:
df['Type'].unique()

array(['Restaurant', 'Attractive', 'Hotel'], dtype=object)

In [27]:
df['Polarity'].value_counts()

Polarity
5.0    136561
4.0     45034
3.0     15519
2.0      5496
1.0      5441
Name: count, dtype: int64

Las clases están desbalanceadas.

In [28]:
df['Type'].value_counts()

Type
Restaurant    86720
Attractive    69921
Hotel         51410
Name: count, dtype: int64

## Procesamiento del lenguaje natural

### Normalización

Para este caso, conviene convertir cada texto a minusculas.

In [29]:
texts_normalized = (df['Title'] + ' ' + df['Review']).str.lower().astype('str')

y_type = df['Type'].replace({'Restaurant': 1, 'Attractive': 2, 'Hotel': 3}).astype(int)
y_pol = df['Polarity'].astype(int)

for text in texts_normalized[:5]:
    print(text)
    print('\n')

mi lugar favorito!!!! excelente lugar para comer y pasar una buena noche!!!
el servicio es de primera y la comida exquisita!!!


lugares interesantes para visitar andar mucho, así que un poco difícil para personas con niños pequeños, pero con mucha historia en la zona, y la diversión de aprender un poco de todo, y explorar las ruinas. la playa también era bastante agradable!


no es el mismo dreams  es nuestra cuarta visita a dreams tulum, elegimos este hotel para festejar mi cumpleaños ya que en este hotel nos comprometimos y casamos y tenemos un cariño muy especial por este lugar, pero mostramos que cambiaron las cosas.  en cuestión de instalaciones sigue perfecto!! la playa muy limpia a pesar del sargazo ( es una cuestión natural incontrolable).   pero en la amabilidad y servicio que los distinguía lo han perdido bastante, los empleados andan corriendo por todos lados, gritando de un lado a otro tratando de organizarse y pasamos varios detalles como por ejemplo mi esposo pidió un ju

C:\Users\Dell\AppData\Local\Temp\ipykernel_18456\3926372241.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_type = df['Type'].replace({'Restaurant': 1, 'Attractive': 2, 'Hotel': 3}).astype(int)


### Tokenización y Lemmatización

Eliminemos los signos de puntuación, palabras vacías y letras, ya que no aportan nada a este análsis.

In [30]:
nlp = spacy.load("es_core_news_sm", disable=["parser", "ner"]) # Desactivar "parser" y "ner" para eficientizar el proceso

docs = list(nlp.pipe(texts_normalized, batch_size=100)) # Procesamiento en lotes

lemmas_by_text = []
for doc in docs:
    lemmas = [token.lemma_ for token in doc if (
        not token.is_stop # Eliminar palabras vacias
        and not token.is_punct # Eliminemos signos de puntuación 
        and token.is_alpha  # Solo letras
    )] 
    lemmas_by_text.append(lemmas)

for lemmas in lemmas_by_text[:5]:
    print(lemmas)

['lugar', 'favorito', 'excelente', 'lugar', 'comer', 'pasar', 'noche', 'servicio', 'comida', 'exquisito']
['lugar', 'interesante', 'visitar', 'andar', 'difícil', 'persona', 'niño', 'pequeño', 'historia', 'zona', 'diversión', 'aprender', 'explorar', 'ruina', 'playa', 'agradable']
['dreams', 'cuarto', 'visita', 'dreams', 'tulum', 'elegir', 'hotel', 'festejar', 'cumpleaños', 'hotel', 'comprometer', 'casamo', 'cariño', 'especial', 'lugar', 'mostrir', 'cambiar', 'cosa', 'cuestión', 'instalación', 'perfecto', 'playa', 'limpio', 'sargazo', 'cuestión', 'natural', 'incontrolable', 'amabilidad', 'servicio', 'distinguir', 'perder', 'empleado', 'andar', 'correr', 'lado', 'gritar', 'tratar', 'organizar él', 'pasar', 'detalle', 'ejemplo', 'esposo', 'pedir', 'juego', 'verde', 'mesera', 'contestar', 'parar', 'esquina', 'llevar', 'café', 'jamás', 'haber', 'dreams', 'topar', 'staff']
['panorama', 'cerca', 'cancãºn', 'estar', 'cancãºn', 'puerto', 'tomar', 'ferry', 'isla', 'mujer', 'despuã', 's', 'corto',

## Partición en entrenamiento y prueba

In [31]:
X_texts = [" ".join(lemmas) for lemmas in lemmas_by_text]
X = pd.Series(X_texts)

In [32]:
X_train, X_test, y_train_pol, y_test_pol, y_train_type, y_test_type = train_test_split(
    X, y_pol, y_type, test_size=0.2, random_state=42)

## Balanceo de clases

Hagamos un muestreo estratificado para balancear las clases.

In [33]:
sample_pol1 = y_train_pol[y_train_pol == 1].sample(4300, random_state=42)
sample_pol2 = y_train_pol[y_train_pol == 2].sample(4300, random_state=42)
sample_pol3 = y_train_pol[y_train_pol == 3].sample(4300, random_state=42)
sample_pol4 = y_train_pol[y_train_pol == 4].sample(4300, random_state=42)
sample_pol5 = y_train_pol[y_train_pol == 5].sample(4300, random_state=42)

indices_samples = pd.concat([
    sample_pol1,
    sample_pol2,
    sample_pol3,
    sample_pol4,
    sample_pol5
]).index

In [34]:
X_train_pol = X_train.loc[indices_samples]
y_train_pol = y_train_pol.loc[indices_samples]

## Vectorización de texto

Transformemos el texto a un formato adecuado.

In [35]:
vectorizer1 = TfidfVectorizer()
X_train_pol_vec = vectorizer1.fit_transform(X_train_pol)
X_test_pol_vec = vectorizer1.transform(X_test)

In [36]:
vectorizer2 = TfidfVectorizer()
X_train_type_vec = vectorizer2.fit_transform(X_train) 
X_test_type_vec = vectorizer2.transform(X_test)

## Modelo de polaridad del sentimiento

Primero probemos con algoritmos de clasificación probabilísticos y lineales. En caso de que tengan un buen desempeño, optemos por estos algoritmos, en otro caso sería buena práctica probar con algoritmos de clasificación no lineales como árboles de decisión, bosques aleatorios o redes neuronales.

### Clasificador bayesiano ingenuo multinomial

#### Contrucción del modelo

In [38]:
model1_pol = MultinomialNB()
model1_pol.fit(X_train_pol_vec, y_train_pol)

MultinomialNB()

#### Evaluación

In [40]:
y_pred1_pol = model1_pol.predict(X_test_pol_vec)

In [41]:
print(classification_report(y_test_pol, y_pred1_pol))

              precision    recall  f1-score   support

           1       0.42      0.65      0.51      1063
           2       0.19      0.42      0.26      1106
           3       0.27      0.34      0.30      3059
           4       0.34      0.55      0.42      9272
           5       0.88      0.61      0.72     27111

    accuracy                           0.57     41611
   macro avg       0.42      0.51      0.44     41611
weighted avg       0.69      0.57      0.60     41611



### Regresión logística multinomial

#### Optimización de hiperparámetros

In [42]:
param_dist2_pol = {'C': [0.1, 1, 10]}
random_search2_pol = GridSearchCV(LogisticRegression(max_iter=1000), param_dist2_pol, cv=5, n_jobs=2, verbose=2)
search2_pol = random_search2_pol.fit(X_train_pol_vec, y_train_pol)
search2_pol.best_params_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


{'C': 1}

#### Construcción del modelo

In [43]:
model2_pol = LogisticRegression(C=1, max_iter=1000)
model2_pol.fit(X_train_pol_vec, y_train_pol)

LogisticRegression(C=1, max_iter=1000)

#### Evaluación

In [44]:
y_pred2_pol = model2_pol.predict(X_test_pol_vec)

In [45]:
print(classification_report(y_test_pol, y_pred2_pol))

              precision    recall  f1-score   support

           1       0.41      0.65      0.50      1063
           2       0.21      0.41      0.27      1106
           3       0.27      0.41      0.33      3059
           4       0.37      0.48      0.41      9272
           5       0.87      0.67      0.76     27111

    accuracy                           0.60     41611
   macro avg       0.42      0.52      0.45     41611
weighted avg       0.68      0.60      0.63     41611



### Árbol de decisión

Ahora probemos con clasificadores no lineales, que deberían de tener un mejor desempeño por la complejidad de los datos con los que estamos trabajando y la cantidad de clases en la variable de respuesta.

#### Optimización de hiperparámetros

In [46]:
param_dist3_pol = {'max_depth': [10, 20, 30, 40], 'min_samples_split': randint(2, 61)}
random_search3_pol = RandomizedSearchCV(DecisionTreeClassifier(), param_dist3_pol, n_iter=10, cv=5, n_jobs=2, verbose=2)
search3_pol = random_search3_pol.fit(X_train_pol_vec, y_train_pol)
search3_pol.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'max_depth': 30, 'min_samples_split': 26}

#### Construcción del modelo

In [47]:
model3_pol = DecisionTreeClassifier(max_depth=30, min_samples_split=28)
model3_pol.fit(X_train_pol_vec, y_train_pol)

DecisionTreeClassifier(max_depth=30, min_samples_split=28)

#### Evaluación

In [48]:
y_pred3_pol = model3_pol.predict(X_test_pol_vec)

In [49]:
print(classification_report(y_test_pol, y_pred3_pol))

              precision    recall  f1-score   support

           1       0.24      0.40      0.30      1063
           2       0.12      0.32      0.18      1106
           3       0.17      0.23      0.19      3059
           4       0.28      0.51      0.36      9272
           5       0.83      0.48      0.61     27111

    accuracy                           0.46     41611
   macro avg       0.33      0.39      0.33     41611
weighted avg       0.63      0.46      0.50     41611



### Bosque aleatorio

#### Optimización de hiperparámetros

In [50]:
param_dist4_pol = {'max_depth': [10, 20, 30, 40], 'min_samples_split': randint(2, 61)}
random_search4_pol = RandomizedSearchCV(RandomForestClassifier(), param_dist4_pol, n_iter=10, cv=5, n_jobs=2, verbose=2)
search4_pol = random_search4_pol.fit(X_train_pol_vec, y_train_pol)
search4_pol.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits



KeyboardInterrupt



#### Construcción del modelo

In [51]:
model4_pol = RandomForestClassifier(max_depth=40, min_samples_split=32)
model4_pol.fit(X_train_pol_vec, y_train_pol)

RandomForestClassifier(max_depth=40, min_samples_split=32)

#### Evaluación

In [52]:
y_pred4_pol = model4_pol.predict(X_test_pol_vec)

In [53]:
print(classification_report(y_test_pol, y_pred4_pol))

              precision    recall  f1-score   support

           1       0.29      0.70      0.41      1063
           2       0.19      0.31      0.23      1106
           3       0.29      0.29      0.29      3059
           4       0.37      0.40      0.38      9272
           5       0.82      0.73      0.78     27111

    accuracy                           0.61     41611
   macro avg       0.39      0.49      0.42     41611
weighted avg       0.65      0.61      0.63     41611



### SVM con kernel gaussiano

#### Construcción del modelo

In [54]:
model5_pol = SVC()
model5_pol.fit(X_train_pol_vec, y_train_pol)

SVC()

#### Evaluación

In [55]:
y_pred5_pol = model5_pol.predict(X_test_pol_vec)

In [56]:
print(classification_report(y_test_pol, y_pred5_pol))

              precision    recall  f1-score   support

           1       0.46      0.64      0.54      1063
           2       0.21      0.43      0.28      1106
           3       0.27      0.42      0.33      3059
           4       0.36      0.51      0.42      9272
           5       0.88      0.64      0.74     27111

    accuracy                           0.59     41611
   macro avg       0.44      0.53      0.46     41611
weighted avg       0.69      0.59      0.62     41611



### Red neuronal recurrente

#### Construcción del modelo

In [58]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_pol)

X_train_seq = tokenizer.texts_to_sequences(X_train_pol)
X_test_seq = tokenizer.texts_to_sequences(X_test)

maxlen = 150
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

In [62]:
model6_pol = Sequential()
model6_pol.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128))
model6_pol.add(LSTM(128, return_sequences=True))
model6_pol.add(Dropout(0.3))
model6_pol.add(LSTM(64))
model6_pol.add(Dropout(0.3))
model6_pol.add(Dense(5, activation='softmax'))

model6_pol.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model6_pol.fit(X_train_pad, y_train_pol-1, batch_size=32, epochs=10, validation_data=(X_test_pad, y_test_pol-1))

Epoch 1/10
672/672 ━━━━━━━━━━━━━━━━━━━━ 380s 560ms/step - accuracy: 0.3878 - loss: 1.3289 - val_accuracy: 0.3874 - val_loss: 1.3080
Epoch 2/10
672/672 ━━━━━━━━━━━━━━━━━━━━ 373s 555ms/step - accuracy: 0.6091 - loss: 0.9207 - val_accuracy: 0.5983 - val_loss: 0.9143
Epoch 3/10
672/672 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.7203 - loss: 0.7204

KeyboardInterrupt: 

#### Evaluación

In [ ]:
model6_pol.evaluate(X_test_pad, y_test_pol-1)

## Modelo: Type

### Clasificador bayesiano ingenuo multinomial

#### Construcción del modelo

In [63]:
model1_type = MultinomialNB()
model1_type.fit(X_train_type_vec, y_train_type)

MultinomialNB()

#### Evaluación

In [64]:
y_pred1_type = model1_type.predict(X_test_type_vec)
print(classification_report(y_test_type, y_pred1_type))

              precision    recall  f1-score   support

           1       0.92      0.96      0.94     17306
           2       0.96      0.94      0.95     14000
           3       0.94      0.89      0.91     10305

    accuracy                           0.94     41611
   macro avg       0.94      0.93      0.93     41611
weighted avg       0.94      0.94      0.94     41611



### Regresión logística multinomial

#### Optimización de hiperparámetros

In [65]:
param_dist2_type = {'C': [0.1, 1, 10]}
random_search2_type = GridSearchCV(LogisticRegression(), param_dist2_type, cv=5, n_jobs=2, verbose=2)
search2_type = random_search2_type.fit(X_train_type_vec, y_train_type)
search2_type.best_params_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


{'C': 1}

#### Construcción del modelo

In [66]:
model2_type = LogisticRegression()
model2_type.fit(X_train_type_vec, y_train_type)

LogisticRegression()

#### Evaluación

In [67]:
y_pred2_type = model2_type.predict(X_test_type_vec)

In [68]:
print(classification_report(y_test_type, y_pred2_type))

              precision    recall  f1-score   support

           1       0.95      0.97      0.96     17306
           2       0.96      0.96      0.96     14000
           3       0.96      0.92      0.94     10305

    accuracy                           0.95     41611
   macro avg       0.96      0.95      0.95     41611
weighted avg       0.95      0.95      0.95     41611



### Árbol de decisión

#### Optimización de hiperparámetros

In [69]:
param_dist3_type = {'max_depth': [10, 20, 30, 40], 'min_samples_split': randint(2, 61)}
random_search3_type = RandomizedSearchCV(DecisionTreeClassifier(), param_dist3_type, n_iter=10, cv=5, verbose=2)
search3_type = random_search3_type.fit(X_train_type_vec, y_train_type)
search3_type.best_params_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


KeyboardInterrupt: 

#### Construcción del modelo

In [ ]:
model3_type = DecisionTreeClassifier(max_depth=10, min_samples_split=42)
model3_type.fit(X_train_type_vec, y_train_type)

In [ ]:
y_pred3_type = model3_type.predict(X_test_type_vec)
print(classification_report(y_test_type, y_pred3_type))

## Guardar modelos

In [70]:
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer2, f)

In [71]:
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [ ]:
model6_pol.save("model1.h5")

In [72]:
with open("model2.pkl", "wb") as f:
    pickle.dump(model2_type, f)